In [1]:
# load packages
import numpy as np
import pandas as pd
import os
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
# 경로 설정
data_dir = '/Users/jaewone/Downloads/floor_noise/sensor_data/data'

noise_data_dir = os.path.join(data_dir, 'noise')
white_data_dir = os.path.join(data_dir, 'white')

### 데이터 가져오기


In [3]:
def get_data(noise_data_dir, white_data_dir, test_data_rate=0.1):
    assert test_data_rate > 0 and test_data_rate < 1 and isinstance(
        test_data_rate, float), 'test_data_rate must be float and 0 < test_data_rate < 1'

    noise_file_list = [os.path.join(noise_data_dir, file)
                       for file in os.listdir(noise_data_dir)]
    white_file_list = [os.path.join(white_data_dir, file)
                       for file in os.listdir(white_data_dir)]

    x = np.concatenate((noise_file_list, white_file_list), axis=0)
    y = np.concatenate(([(0, 1) for _ in range(len(noise_file_list))], [
                       (1, 0) for _ in range(len(white_file_list))]), axis=0)

    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=test_data_rate, random_state=42)

    x_train = np.array(
        [pd.read_csv(file_path).values for file_path in x_train])
    x_train = x_train.reshape(*x_train.shape, 1)
    x_test = np.array([pd.read_csv(file_path).values for file_path in x_test])
    x_test = x_test.reshape(*x_test.shape, 1)

    return x_train, x_test, y_train, y_test


x_train, x_test, y_train, y_test = get_data(
    noise_data_dir, white_data_dir, test_data_rate=0.1)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((540, 100, 3, 1), (540, 2), (60, 100, 3, 1), (60, 2))

In [4]:
def quantized(data):
    """
    값을 0과 1사이로 정규화한 다음 -128에서 127 사이의 int8 값으로 양자화한다.
    """
    SENSOR_MAX = 1.0
    SENSOR_MIN = -2.0

    normalized_values = (data - SENSOR_MIN) / \
        (SENSOR_MAX - SENSOR_MIN) * 2.0 - 1.0
    return np.clip(np.round(normalized_values * 127), -128, 127).astype(np.int8)


x_train = quantized(x_train)
x_test = quantized(x_test)

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((540, 100, 3, 1), (540, 2), (60, 100, 3, 1), (60, 2))

In [5]:
# x_train = np.array([pd.read_csv(file_path).values for file_path in ['/Users/jaewone/Downloads/floor_noise/sensor_data/data/white/white_8.csv']])
# x_train = np.array([pd.read_csv(file_path).values for file_path in ['/Users/jaewone/Downloads/floor_noise/sensor_data/data/noise/noise_8.csv']])

# SENSOR_MAX = 1.0
# SENSOR_MIN = -2.0

# normalized_values = (x_train - SENSOR_MIN) / (SENSOR_MAX - SENSOR_MIN) * 2.0 - 1.0
# # print(normalized_values)

# # 정규화된 값을 -128에서 127 사이의 int8 값으로 양자화합니다.
# quantized_values = np.clip(
#     np.round(normalized_values * 127), -128, 127).astype(np.int8)

# value, freq = np.unique(quantized_values, return_counts=True)
# value = np.array(value)
# freq = np.array(freq)

# import matplotlib.pyplot as plt

# for i in range(6):
#   idx = freq.argmax()
#   value = np.delete(value, idx)
#   freq = np.delete(freq, idx)

# plt.bar(value, freq)

### 모델 구성


In [6]:
# 3. 모델 구성
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(8, (3, 4), activation='relu', input_shape=(
        100, 3, 1), padding='same'),  # (batch, 100, 3, 8)
    tf.keras.layers.MaxPool2D((3, 3)),  # (batch, 33, 1, 8)
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Conv2D(16, (4, 1), padding="same",
                           activation="relu"),  # (batch, 42, 1, 16)
    tf.keras.layers.MaxPool2D((3, 1), padding="same"),  # (batch, 14, 1, 16)
    tf.keras.layers.Dropout(0.1),  # (batch, 14, 1, 16)
    tf.keras.layers.Flatten(),  # (batch, 176)
    tf.keras.layers.Dense(16, activation="relu"),  # (batch, 8)
    tf.keras.layers.Dense(2, activation='softmax'),  # (batch, 1)
])
model.summary()

Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 3, 8)         104       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 33, 1, 8)         0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 33, 1, 8)          0         
                                                                 
 conv2d_1 (Conv2D)           (None, 33, 1, 16)         528       
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 11, 1, 16)        0         
 2D)                                                             
                                             

2023-11-06 20:14:33.467968: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-11-06 20:14:33.468072: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 모델 학습


In [7]:
model.compile(optimizer='adam', loss='binary_crossentropy',
              metrics=['accuracy'])

# 4. 모델 학습
model.fit(x_train, y_train, epochs=100, batch_size=32,
          validation_data=(x_test, y_test))

Epoch 1/100


2023-11-06 20:14:33.696320: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-11-06 20:14:33.929294: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 1s 26ms/step - loss: 1.3240 - accuracy: 0.4463 - val_loss: 0.7775 - val_accuracy: 0.4833
Epoch 2/100
 8/17 [=============>................] - ETA: 0s - loss: 0.9553 - accuracy: 0.4609

2023-11-06 20:14:34.506014: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


17/17 [==============================] - 0s 9ms/step - loss: 0.9017 - accuracy: 0.4593 - val_loss: 0.7131 - val_accuracy: 0.5000
Epoch 3/100
17/17 [==============================] - 0s 10ms/step - loss: 0.7818 - accuracy: 0.4981 - val_loss: 0.6641 - val_accuracy: 0.5000
Epoch 4/100
17/17 [==============================] - 0s 9ms/step - loss: 0.7195 - accuracy: 0.5111 - val_loss: 0.6990 - val_accuracy: 0.4167
Epoch 5/100
17/17 [==============================] - 0s 9ms/step - loss: 0.7087 - accuracy: 0.5426 - val_loss: 0.6639 - val_accuracy: 0.4833
Epoch 6/100
17/17 [==============================] - 0s 10ms/step - loss: 0.6842 - accuracy: 0.5685 - val_loss: 0.6730 - val_accuracy: 0.4333
Epoch 7/100
17/17 [==============================] - 0s 10ms/step - loss: 0.6772 - accuracy: 0.6185 - val_loss: 0.7062 - val_accuracy: 0.4167
Epoch 8/100
17/17 [==============================] - 0s 12ms/step - loss: 0.6533 - accuracy: 0.6426 - val_loss: 0.7449 - val_accuracy: 0.4167
Epoch 9/100
17/17 [==

### 예측 및 모델 평가


In [8]:
# predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

y_pred_max = np.argmax(model.predict(x_test), axis=1)
y_test_max = np.argmax(y_test, axis=1)
y_pred_max_sum = np.sum(y_pred_max)

confusion_matrix(y_test_max, y_pred_max)
print('accuracy: ', np.sum(y_test_max == y_pred_max) / len(y_test_max))
print('precision: ', np.sum(y_test_max * y_pred_max == 1) /
      y_pred_max_sum if y_pred_max_sum != 0 else 0)
print('recall: ', np.sum(y_test_max * y_pred_max == 1) / np.sum(y_test_max))
print('f1-score: ', f1_score(y_test_max, y_pred_max))

accuracy:  1.0
precision:  1.0
recall:  1.0
f1-score:  1.0


2023-11-06 20:14:52.046379: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [9]:
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import f1_score

# y_pred = model.predict(x_test).flatten()
# y_pred = np.where(y_pred > 0.5, 1, 0)
# y_pred_sum = np.sum(y_pred)

# confusion_matrix(y_test, y_pred)
# print('accuracy: ', np.sum(y_test == y_pred) / len(y_test))
# print('precision: ', np.sum(y_test * y_pred == 1) /
#       y_pred_sum if y_pred_sum != 0 else 0)
# print('recall: ', np.sum(y_test * y_pred == 1) / np.sum(y_test))
# print('f1-score: ', f1_score(y_test, y_pred))

### tflite로 저장


In [10]:
# save tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)


def representative_data_gen():
    for _ in range(100):
        data = np.random.rand(1, 100, 3, 1)
        yield [data.astype(np.float32)]


converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.float32

tflite_model = converter.convert()
open("model.tflite", "wb").write(tflite_model)

2023-11-06 20:14:52.295637: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /var/folders/10/9_p98m6j42n84y4wmf_k0krc0000gn/T/tmpvs8huitg/assets


/Users/jaewone/ENTER/envs/tf25/lib/python3.9/site-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "
2023-11-06 20:14:55.022078: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2023-11-06 20:14:55.022090: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2023-11-06 20:14:55.022520: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/10/9_p98m6j42n84y4wmf_k0krc0000gn/T/tmpvs8huitg
2023-11-06 20:14:55.023597: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2023-11-06 20:14:55.023603: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /var/folders/10/9_p98m6j42n84y4wmf_k0krc0000gn/T/tmpvs8huitg
2023-11-06 20:14:55.027638

7920

In [11]:
# 저장된 tflite 파일 테스트
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_details, output_details

([{'name': 'serving_default_conv2d_input:0',
   'index': 0,
   'shape': array([  1, 100,   3,   1], dtype=int32),
   'shape_signature': array([ -1, 100,   3,   1], dtype=int32),
   'dtype': numpy.int8,
   'quantization': (0.003921520430594683, -128),
   'quantization_parameters': {'scales': array([0.00392152], dtype=float32),
    'zero_points': array([-128], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}],
 [{'name': 'StatefulPartitionedCall:0',
   'index': 18,
   'shape': array([1, 2], dtype=int32),
   'shape_signature': array([-1,  2], dtype=int32),
   'dtype': numpy.float32,
   'quantization': (0.0, 0),
   'quantization_parameters': {'scales': array([], dtype=float32),
    'zero_points': array([], dtype=int32),
    'quantized_dimension': 0},
   'sparsity_parameters': {}}])

In [12]:
input_shape = input_details[0]['shape']
# input_data = np.array(np.random.random_sample(input_shape), dtype=np.int8)
input_data = [x_test[10]]
interpreter.set_tensor(input_details[0]['index'], input_data)

interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)

[[0.50390625 0.49609375]]


바이트 파일로 변환하는건 windows에서는 직접적으로 안됨.

tflite 파일 보내주면 필자가 변환함.


### 바이트 파일로 변환


In [13]:
# C source file를 만들기 위한 xxd 라이브러리 설치
# !apt-get -qq install xxd

In [14]:
# 모델을 C 소스 파일로 변경한다.
!xxd -i model.tflite > model.cc

In [15]:
# C 소스 파일을 출력한다.
!cat model.cc

unsigned char model_tflite[] = {
  0x1c, 0x00, 0x00, 0x00, 0x54, 0x46, 0x4c, 0x33, 0x14, 0x00, 0x20, 0x00,
  0x04, 0x00, 0x08, 0x00, 0x0c, 0x00, 0x10, 0x00, 0x14, 0x00, 0x00, 0x00,
  0x18, 0x00, 0x1c, 0x00, 0x14, 0x00, 0x00, 0x00, 0x03, 0x00, 0x00, 0x00,
  0x78, 0x00, 0x00, 0x00, 0x6c, 0x00, 0x00, 0x00, 0x60, 0x01, 0x00, 0x00,
  0x0c, 0x00, 0x00, 0x00, 0x08, 0x01, 0x00, 0x00, 0x80, 0x00, 0x00, 0x00,
  0x15, 0x00, 0x00, 0x00, 0xac, 0x1e, 0x00, 0x00, 0xa8, 0x1e, 0x00, 0x00,
  0x90, 0x1d, 0x00, 0x00, 0x54, 0x1c, 0x00, 0x00, 0x7c, 0x1b, 0x00, 0x00,
  0x4c, 0x18, 0x00, 0x00, 0xf0, 0x16, 0x00, 0x00, 0x80, 0x0b, 0x00, 0x00,
  0xdc, 0x0a, 0x00, 0x00, 0x44, 0x0a, 0x00, 0x00, 0xd8, 0x09, 0x00, 0x00,
  0x80, 0x1e, 0x00, 0x00, 0x7c, 0x1e, 0x00, 0x00, 0x78, 0x1e, 0x00, 0x00,
  0x74, 0x1e, 0x00, 0x00, 0x70, 0x1e, 0x00, 0x00, 0x6c, 0x1e, 0x00, 0x00,
  0x68, 0x1e, 0x00, 0x00, 0x64, 0x1e, 0x00, 0x00, 0x60, 0x1e, 0x00, 0x00,
  0xe0, 0x00, 0x00, 0x00, 0x01, 0x00, 0x00, 0x00, 0x18, 0x01, 0x00, 0x00,
  0x0